BROADCASTING:
describe cómo NumPy trata matrices con diferentes formas durante operaciones aritméticas. Sujeto a ciertas restricciones, la matriz más pequeña se "difunde" a través de la matriz más grande para que tengan formas compatibles. La transmisión proporciona un medio para vectorizar operaciones de matrices para que el bucle se produzca en C en lugar de Python. Lo hace sin realizar copias innecesarias de datos y, por lo general, conduce a implementaciones de algoritmos eficientes. Sin embargo, hay casos en los que la transmisión es una mala idea porque conduce a un uso ineficiente de la memoria que ralentiza el cálculo.

Las operaciones NumPy generalmente se realizan en pares de matrices elemento por elemento. En el caso más simple, las dos matrices deben tener exactamente la misma forma, como en el siguiente ejemplo:

In [2]:
import numpy as np

a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])
print("a = np.array([1.0, 2.0, 3.0])")
print("b = np.array([2.0, 2.0, 2.0])")
print()
print(f"a * b = {a * b}") 


a = np.array([1.0, 2.0, 3.0])
b = np.array([2.0, 2.0, 2.0])

a * b = [2. 4. 6.]


La regla de transmisión de NumPy relaja esta restricción cuando las formas de las matrices cumplen ciertas restricciones. El ejemplo de transmisión más simple ocurre cuando se combinan una matriz y un valor escalar en una operación:

In [3]:
a = np.array([1.0, 2.0, 3.0])
print("a = np.array([1.0, 2.0, 3.0])")
print("b = 2.0")
print()
print(f"a * b = {a * b}")


a = np.array([1.0, 2.0, 3.0])
b = 2.0

a * b = [2. 4. 6.]


El resultado es equivalente al ejemplo anterior donde 'b' había una matriz. Podemos imaginar que el escalar bse estira durante la operación aritmética hasta formar una matriz con la misma forma que 'a'. Los nuevos elementos en 'b', son simplemente copias del escalar original. La analogía del estiramiento es sólo conceptual. NumPy es lo suficientemente inteligente como para usar el valor escalar original sin hacer copias, de modo que las operaciones de transmisión sean lo más eficientes posible en memoria y computación.

El código del segundo ejemplo es más eficiente que el del primero porque la transmisión mueve menos memoria durante la multiplicación ( 'b' es un escalar en lugar de una matriz).

NORMAS GENERALES DEL BROADCASTING   

Cuando opera en dos matrices, NumPy compara sus formas por elementos. Comienza con la dimensión final (es decir, la más a la derecha) y avanza hacia la izquierda. Dos dimensiones son compatibles cuando

1. Son iguales, o

2. Uno de ellos es 1.

Si no se cumplen estas condiciones, se genera una excepción que indica que las matrices tienen formas incompatibles.ValueError: operands could not be broadcast together

No es necesario que las matrices de entrada tengan la misma cantidad de dimensiones. La matriz resultante tendrá la misma cantidad de dimensiones que la matriz de entrada con la mayor cantidad de dimensiones, donde el tamaño de cada dimensión es el tamaño más grande de la dimensión correspondiente entre las matrices de entrada. Tenga en cuenta que se supone que las dimensiones que faltan tienen el tamaño uno.

Por ejemplo, si tiene una 256x256x3 matriz de valores RGB y desea escalar cada color de la imagen con un valor diferente, puede multiplicar la imagen por una matriz unidimensional con 3 valores. Alinear los tamaños de los ejes posteriores de estos arreglos de acuerdo con las reglas de transmisión muestra que son compatibles:

Image  (3d array): 256 x 256 x 3
Scale  (1d array):             3
Result (3d array): 256 x 256 x 3

Cuando alguna de las dimensiones comparadas es una, se utiliza la otra. En otras palabras, las dimensiones con tamaño 1 se estiran o “copian” para que coincidan con las otras.

En el siguiente ejemplo, tanto las matrices Acomo Btienen ejes con longitud uno que se expanden a un tamaño mayor durante la operación de transmisión:

A      (4d array):  8 x 1 x 6 x 1
B      (3d array):      7 x 1 x 5
Result (4d array):  8 x 7 x 6 x 5

Matrices BroadCasting:

Un conjunto de matrices se denomina "BroadCasting" a la misma forma si las reglas anteriores producen un resultado válido.

Por ejemplo, si a.shape es (5,1), b.shape es (1,6), c.shape es (6,) y d.shape es () de modo que 'd' es un escalar, entonces a , b , c y d se pueden transmitir a la dimensión (5 ,6); y

- 'a' actúa como una matriz (5,6) donde a[:,0]se transmite a las otras columnas,

- 'b' actúa como una matriz (5,6) donde b[0,:]se transmite a las otras filas,

- 'c' actúa como una matriz (1,6) y por lo tanto como una matriz (5,6) donde c[:] se transmite a cada fila, y finalmente,

- 'd' actúa como una matriz (5,6) donde se repite el valor único.

Aquí hay algunos ejemplos más:

A      (2d array):  5 x 4
B      (1d array):      1
Result (2d array):  5 x 4

A      (2d array):  5 x 4
B      (1d array):      4
Result (2d array):  5 x 4

A      (3d array):  15 x 3 x 5
B      (3d array):  15 x 1 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 5
Result (3d array):  15 x 3 x 5

A      (3d array):  15 x 3 x 5
B      (2d array):       3 x 1
Result (3d array):  15 x 3 x 5

A continuación se muestran ejemplos de formas que no se transmiten:

A      (1d array):  3
B      (1d array):  4 # trailing dimensions do not match

A      (2d array):      2 x 1
B      (3d array):  8 x 4 x 3 # second from last dimensions mismatched

Un ejemplo de transmisión cuando se agrega una matriz unidimensional a una matriz bidimensional:

In [4]:
a = np.array([[ 0.0,  0.0,  0.0],
              [10.0, 10.0, 10.0],
              [20.0, 20.0, 20.0],
              [30.0, 30.0, 30.0]])

b = np.array([1.0, 2.0, 3.0])

print(f"a + b = {a + b}")

b = np.array([1.0, 2.0, 3.0, 4.0])

print(f"a + b = {a + b}")

a + b = [[ 1.  2.  3.]
 [11. 12. 13.]
 [21. 22. 23.]
 [31. 32. 33.]]


ValueError: operands could not be broadcast together with shapes (4,3) (4,) 

Una matriz unidimensional agregada a una matriz bidimensional da como resultado una transmisión si el número de elementos de la matriz unidimensional coincide con el número de columnas de la matriz bidimensional.

Cuando las dimensiones finales de las matrices son desiguales, la transmisión falla porque es imposible alinear los valores en las filas de la primera matriz con los elementos de la segunda matriz para la suma elemento por elemento.

El 'broadcasting' proporciona una manera conveniente de tomar el producto exterior (o cualquier otra operación exterior) de dos matrices. El siguiente ejemplo muestra una operación de suma externa de dos matrices unidimensionales:

In [5]:
a = np.array([0.0, 10.0, 20.0, 30.0])
b = np.array([1.0, 2.0, 3.0])
print(f"a[:, np.newaxis] + b = {a[:, np.newaxis] + b}") 

a[:, np.newaxis] + b = [[ 1.  2.  3.]
 [11. 12. 13.]
 [21. 22. 23.]
 [31. 32. 33.]]


En algunos casos, la difusión extiende ambas matrices para formar una matriz de salida más grande que cualquiera de las matrices iniciales.

Aquí el 'newaxis' operador de índice inserta un nuevo eje en 'a', convirtiéndolo en una matriz bidimensional 4x1. Combinando la 4x1matriz con 'b', que tiene forma (3,), se obtiene una 4x3 matriz.

EJEMPLO PRACTICO: CUANTIFICACION VECTORIAL

El 'Broadcasting' surge con bastante frecuencia en los problemas del mundo real. Un ejemplo típico ocurre en el algoritmo de cuantificación vectorial (VQ) utilizado en teoría de la información, clasificación y otras áreas relacionadas. La operación básica en VQ encuentra el punto más cercano en un conjunto de puntos, llamado codesen la jerga VQ, a un punto dado, llamado observation. En el caso bidimensional muy simple que se muestra a continuación, los valores observationdescriben el peso y la altura de un atleta que se va a clasificar. Representan codesdiferentes clases de atletas. [ 1 ] Encontrar el punto más cercano requiere calcular la distancia entre la observación y cada uno de los códigos. La distancia más corta proporciona la mejor coincidencia. En este ejemplo, codes[0] la clase más cercana indica que el atleta probablemente sea un jugador de baloncesto.

In [6]:
from numpy import array, argmin, sqrt, sum

observation = array([111.0, 188.0])
codes = array([[102.0, 203.0],
               [132.0, 193.0],
               [45.0, 155.0],
               [57.0, 173.0]])

diff = codes - observation    # the broadcast happens here

dist = sqrt(sum(diff**2,axis=-1))

print(f"argmin(dist) = {argmin(dist)}")

argmin(dist) = 0


En este ejemplo, la 'observation' matriz se estira para que coincida con la forma de la codesmatriz:

Observation      (1d array):      2
Codes            (2d array):  4 x 2
Diff             (2d array):  4 x 2

Normalmente, una gran cantidad de archivos observations, quizás leídos de una base de datos, se comparan con un conjunto de archivos codes. Considere este escenario:

Observation      (2d array):      10 x 3
Codes            (3d array):   5 x 1 x 3
Diff             (3d array):  5 x 10 x 3

La matriz tridimensional, diff, es una consecuencia de la radiodifusión, no una necesidad para el cálculo. Grandes conjuntos de datos generarán una gran matriz intermedia que es computacionalmente ineficiente. En cambio, si cada observación se calcula individualmente usando un bucle de Python alrededor del código en el ejemplo bidimensional anterior, se usa una matriz mucho más pequeña.

La transmisión es una herramienta poderosa para escribir código breve y generalmente intuitivo que realiza sus cálculos de manera muy eficiente en C. Sin embargo, hay casos en los que la transmisión utiliza cantidades innecesariamente grandes de memoria para un algoritmo en particular. En estos casos, es mejor escribir el bucle externo del algoritmo en Python. Esto también puede producir un código más legible, ya que los algoritmos que utilizan la transmisión tienden a volverse más difíciles de interpretar a medida que aumenta el número de dimensiones en la transmisión.